# Regras de Associação
Aluna: Maria Eduarda Guedes Pinto Gianisella


# Exercício 1

In [29]:
from itertools import combinations

transacoes = [
    ['tree', 'cup', 'paper'],
    ['book', 'tomato', 'pen'],
    ['pen', 'book', 'tree'],
    ['tomato', 'pen', 'cup'],
    ['tree', 'paper', 'book'],
    ['pen', 'book']
]

def calcular_suporte(conjunto, transacoes):
    return sum(1 for t in transacoes if set(conjunto).issubset(t)) / len(transacoes) * 100

itens_distintos = set(item for transacao in transacoes for item in transacao)

conjuntos_suporte = {conjunto: calcular_suporte(conjunto, transacoes) for tamanho in range(2, len(itens_distintos) + 1) for conjunto in combinations(itens_distintos, tamanho) if calcular_suporte(conjunto, transacoes) >= 33}

for conjunto, suporte in conjuntos_suporte.items():
    print(f"{conjunto}: {suporte:.2f}%")


('paper', 'tree'): 33.33%
('tree', 'book'): 33.33%
('tomato', 'pen'): 33.33%
('pen', 'book'): 50.00%


# Exercício 2

In [33]:
from collections import defaultdict

def gerar_regras(transacoes, confianca_minima, numeros_alvo):
    regras = defaultdict(list)
    itens_distintos = set(item for transacao in transacoes for item in transacao)
    
    suportes_individuais = {item: calcular_suporte({item}, transacoes) for item in itens_distintos}
    
    for tamanho in range(2, len(itens_distintos) + 1):
        for conjunto in combinations(itens_distintos, tamanho):
            if set(conjunto).issubset(numeros_alvo):
                suporte_conjunto = calcular_suporte(set(conjunto), transacoes)
                for item in conjunto:
                    confianca = suporte_conjunto / suportes_individuais[item]
                    if confianca >= confianca_minima:
                        regras[item].append((set(conjunto) - {item}, confianca))
    
    return regras

def imprimir_regras(regras_geradas):
    print("Regras com confiança >= 73%")
    for item, regras in regras_geradas.items():
        for regra in regras:
            print(f"{item} -> {regra[0]} : {regra[1]*100:.2f}%")

transacoes = [
    {1, 3, 5, 7, 8},
    {3, 4, 5, 7},
    {2, 3, 5, 6},
    {2, 4, 5, 8, 9},
    {2, 3, 5, 11},
    {1, 2, 3, 7, 9}
]

numeros_alvo = {2, 3, 5, 7}
confianca_minima = 0.73

regras_geradas = gerar_regras(transacoes, confianca_minima, numeros_alvo)
imprimir_regras(regras_geradas)

Regras com confiança >= 73%
2 -> {3} : 75.00%
2 -> {5} : 75.00%
3 -> {5} : 80.00%
5 -> {3} : 80.00%
7 -> {3} : 100.00%


# Exercício 3

In [41]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

with open('supermercado.csv', 'r') as arquivo:
    data = arquivo.readlines()

transacoes = [linha.strip().split(',') for linha in data]
transacoes = [[str(item) for item in transaction] for transaction in transacoes]

te = TransactionEncoder()
te_array = te.fit(transacoes).transform(transacoes)

df_codificado = pd.DataFrame(te_array, columns=te.columns_)

itemsets_frequentes = apriori(df_codificado, min_support=0.001, use_colnames=True)
regras = association_rules(itemsets_frequentes, metric='confidence', min_threshold=0.25)
regras_bottled_beer = regras[regras['consequents'].apply(lambda x: 'bottled beer' in str(x))]

print("Regras que contêm 'bottled beer' no consequente:")
for indice, linha in regras_bottled_beer.iterrows():
    antecedentes = ', '.join(list(linha['antecedents']))
    consequentes = ', '.join(list(linha['consequents']))
    print(f"Regra: {antecedentes} -> {consequentes} | Suporte: {linha['support']:.4f} | Confiança: {linha['confidence']:.4f} | Lift: {linha['lift']:.4f}")


Regras que contêm 'bottled beer' no consequente:
Regra: liquor -> bottled beer | Suporte: 0.0047 | Confiança: 0.4220 | Lift: 5.2406
Regra: red/blush wine -> bottled beer | Suporte: 0.0049 | Confiança: 0.2540 | Lift: 3.1538
Regra: tea -> bottled beer | Suporte: 0.0011 | Confiança: 0.2895 | Lift: 3.5947
Regra: bottled water, herbs -> bottled beer | Suporte: 0.0012 | Confiança: 0.4000 | Lift: 4.9672
Regra: oil, bottled water -> bottled beer | Suporte: 0.0012 | Confiança: 0.2927 | Lift: 3.6345
Regra: bottled water, sugar -> bottled beer | Suporte: 0.0012 | Confiança: 0.2553 | Lift: 3.1705
Regra: onions, butter -> bottled beer | Suporte: 0.0010 | Confiança: 0.2500 | Lift: 3.1045
Regra: liquor, red/blush wine -> bottled beer | Suporte: 0.0019 | Confiança: 0.9048 | Lift: 11.2353
Regra: liquor, soda -> bottled beer | Suporte: 0.0012 | Confiança: 0.5714 | Lift: 7.0960
Regra: other vegetables, red/blush wine -> bottled beer | Suporte: 0.0015 | Confiança: 0.3061 | Lift: 3.8014
Regra: red/blush wi

# Avaliação dos resultados

Analisando todas as regras de associação geradas a partir dos dados, é possível observar que as regras de associação revelam padrões de compra dos clientes em um supermercado. Elas oferecem oportunidades de decidir como distribuir os produtos nas prateleiras, por exemplo. Ao compreender essas regras, os mercados podem personalizar ofertas e tomar decisões estratégicas para impulsionar as vendas e melhorar a experiência do cliente.

As regras que se destacam são:

**liquor, red/blush wine -> bottled beer**

Alta confiança e lift, indicando uma forte associação entre a compra de licor, vinho tinto/rosé e cerveja engarrafada.

**red/blush wine -> bottled beer** e **liquor -> bottled beer**

Embora tenham uma confiança um pouco menor do que a regra que associa os três intens, ainda indica uma associação relevante entre a compra de vinho tinto/rosé e cerveja engarrafada ou de licor e cerveja engarrafada.

**liquor, soda -> bottled beer**

Também indica uma associação alta entre a compra de licor, soda e cerveja engarrafada juntos. Pode indicar que o cliente planeja preparar drinks ou quer fazer uma compra diversificada de bebidas com uma opção não alcóolica.

**bottled water, herbs -> bottled beer**

Esta regra associa a compra de água engarrafada e ervas com a compra de cerveja engarrafada. Pode indicar padrões de compra específicos para clientes que planejam cozinhar em casa e acompanhar a refeição com uma bebida alcoólica.

**citrus fruit, bottled water, whole milk -> bottled beer**

Esta regra destaca uma combinação de itens que inclui frutas cítricas, água engarrafada, leite integral e cerveja engarrafada. Isso sugere que os clientes estão planejando uma refeição completa e incluem uma bebida alcoólica como parte dela.